<a href="https://colab.research.google.com/github/katrinag2004/dsproject2/blob/main/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas


In [ ]:
from flask import Flask, request, jsonify
import requests
import pandas as pd

app = Flask(__name__)



# Function to get artist information from the Spotify API
def get_spotify_artist_info(artist_name, access_token):
    # The base URL for the Spotify Web API
    url = f'https://api.spotify.com/v1/search?q={artist_name}&type=artist'

    # Set the headers with your access token
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    # Send the request to the Spotify API
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        if data['artists']['items']:
            artist_info = data['artists']['items'][0]  # Get the first matching artist
            return {
                'name': artist_info['name'],
                'popularity': artist_info['popularity'],
                'genres': artist_info['genres'],
                'followers': artist_info['followers']['total']
            }
        else:
            return {'error': 'Artist not found.'}
    else:
        return {'error': 'Failed to retrieve data from Spotify.'}


# Function to load local data
def load_data():
    df = pd.read_csv('recipes.csv')  # Or other local data sources
    return df

# Main route to test if Flask is running
@app.route('/')
def home():
    return "Welcome to the Flask Chatbot!"

# Chatbot query route
@app.route('/ask', methods=['GET'])
def ask():
    question = request.args.get('question')
    if 'weather' in question.lower():
        city = request.args.get('city', 'London')  # Default city
        weather = get_weather(city)
        return jsonify({"answer": f"The weather in {city} is {weather}."})
    elif 'recipe' in question.lower():
        df = load_data()
        recipe = df[df['name'].str.contains('chicken')].iloc[0]  # Example query
        return jsonify({"answer": f"Try this recipe: {recipe['name']}."})
    else:
        return jsonify({"answer": "Sorry, I don't have an answer to that."})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
